In [ ]:
!pip install pinecone-client
!pip install protoc_gen_openapiv2
!pip install -q -U google-generativeai
!pip install tiktoken

In [ ]:
import google.generativeai as genai
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from typing import List, Dict
import tiktoken

In [5]:
genai.configure(api_key='Your API KEY')
pc = Pinecone(api_key='Your API KEY')
model = genai.GenerativeModel('gemini-1.0-pro')

In [6]:
MAX_TOKENS = 225
PINECONE_INDEX_NAME = 'chat-index'


In [7]:
history = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the best way to start learning Python? | Bot: Start with online resources like Codecademy or Coursera. They offer great introductory courses.",
    "3: User: Sounds good! What about practice projects? | Bot: You can build a simple calculator or a to-do list app to practice your skills.",
    "4: User: I'll give that a try. Any good resources for learning web development? | Bot: FreeCodeCamp and Mozilla's MDN Web Docs are excellent places to start.",
    "5: User: Great! What should I focus on first? | Bot: Begin with HTML and CSS, then move on to JavaScript for interactive elements.",
    "6: User: Thanks! What about backend development? | Bot: Learn about server-side languages like Python with Flask or Django, or Node.js with Express.",
    "7: User: I'll look into those. Any tips for debugging code? | Bot: Use tools like the Chrome DevTools for front-end and pdb for Python debugging.",
    "8: User: Those sound helpful. What about version control? | Bot: Git is essential. Platforms like GitHub or GitLab are great for managing your code.",
    "9: User: I'll start using Git. Any good tech blogs or websites? | Bot: Sites like TechCrunch, Hacker News, and Medium have valuable tech content.",
    "10: User: I'll check them out. What about staying updated with tech news? | Bot: Subscribe to newsletters like The Verge, Wired, or Ars Technica.",
    "11: User: Good idea. What are some popular tech podcasts? | Bot: 'Reply All,' 'The Vergecast,' and 'Coding Blocks' are popular tech podcasts.",
    "12: User: I'll listen to those. Any recommendations for open-source projects to contribute to? | Bot: Look for projects on GitHub tagged with 'good first issue' to start contributing.",
    "13: User: I'll find some to contribute to. What are some must-know algorithms? | Bot: Understand sorting algorithms like QuickSort and MergeSort, and data structures like linked lists and hash tables.",
    "14: User: I'll study those. Any tips for coding interviews? | Bot: Practice problems on LeetCode or HackerRank, and understand the basics of data structures and algorithms.",
    "15: User: I'll prepare with those resources. What about learning DevOps? | Bot: Learn about Docker, Kubernetes, and CI/CD tools like Jenkins.",
    "16: User: Those seem important. Any good books on software engineering? | Bot: 'Clean Code' by Robert C. Martin and 'The Pragmatic Programmer' by Andrew Hunt and David Thomas are highly recommended.",
    "17: User: I'll read those. What about good practices for remote work? | Bot: Maintain a regular schedule, use tools like Slack for communication, and have a dedicated workspace.",
    "18: User: Those are great tips. What about staying productive? | Bot: Use techniques like Pomodoro, and tools like Trello or Asana to manage tasks.",
    "19: User: I'll try those out. Any last advice for a tech career? | Bot: Keep learning, stay curious, and network with other professionals in the field.",
    "20: User: Thanks for all the advice! I'll make the most out of it. | Bot: You're welcome! Enjoy your journey and best of luck with your tech career!"
]

In [8]:
# Function to generate embeddings
def generate_embedding(text: str):
        result = genai.embed_content(
          model="models/text-embedding-004",
          content=text,
          task_type="retrieval_document",
          title="Embedding of single string")
        return result['embedding']


In [ ]:
def add_embeddings_to_pinecone(history, index_name='chat-history'):
    """
    This function should:
    1. Encode each message in the history using a transformer model.
    2. Include the message number as part of the embedding.
    3. Create a Pinecone index if it does not exist.
    4. Upsert the encoded messages into the Pinecone index.
    """


    # Create Pinecone index if it does not exist
    index_list = pc.list_indexes().get('indexes', [])
    chat_history_exists = any(index['name'] == 'chat-history' for index in index_list)
    if not chat_history_exists:
        pc.create_index(index_name,
            dimension=768,
            metric="cosine",
            spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        ))  # Assuming embeddings are 768-dimensional
    index = pc.Index(index_name)



    # Parse history and encode messages
    vectors = []
    for item in history:
        # Split user and bot messages
        message_number, conversation = item.split(": ", 1)
        user_message, bot_message = conversation.split(" | Bot: ")
        user_message = user_message.replace("User: ", "")

        # Encode messages
        user_embedding = generate_embedding(user_message)
        bot_embedding = generate_embedding(bot_message)

        # Create metadata
        user_metadata = {"role": "user", "content": user_message, "message_number": message_number}
        bot_metadata = {"role": "assistant", "content": bot_message, "message_number": message_number}

        # Append to vectors
        vectors.append({"id": f"user-{message_number}", "values": user_embedding, "metadata": user_metadata})
        vectors.append({"id": f"bot-{message_number}", "values": bot_embedding, "metadata": bot_metadata})

    # Upsert vectors into Pinecone
    index.upsert(vectors, namespace="history")


add_embeddings_to_pinecone(history)


In [10]:
def retrieve_relevant_history(query,index_name='chat-history'):
    """
    This function should:
    1. Encode the query using a transformer model.
    2. Query the Pinecone index with the encoded query to retrieve the most relevant messages.
    3. Return the relevant messages.
    """

    #Initialize pinecone Index
    index = pc.Index('chat-history')

    #Generate embeddings for query
    query_embedding = generate_embedding(query)

    # Query Pinecone index for relevant messages
    query_response = index.query(namespace='history',vector=query_embedding, top_k=5, include_metadata=True)


    # Extract and return relevant messages
    relevant_messages = []
    for match in query_response['matches']:
        relevant_messages.append(match['metadata']['content'])
    return relevant_messages



In [11]:
def prepare_prompt(test_prompt, index_name='chat-history'):
    """
    This function should:
    1. Retrieve relevant history messages using the RAG mechanism.
    2. Combine the retrieved messages with the test prompt.
    3. Ensure the combined prompt does not exceed the maximum token limit.
    4. Return the combined prompt and the context referred.
    """
    # Retrieve relevant history
    relevant_history = retrieve_relevant_history(test_prompt, index_name=index_name)

    # Combine the retrieved messages with the test prompt
    combined_prompt = "\n".join(relevant_history) + "\n" + test_prompt

    encoding = tiktoken.get_encoding("cl100k_base")
    tokens = encoding.encode(combined_prompt)

    if len(tokens) > MAX_TOKENS:
      # Truncate the combined prompt to fit within the token limit
      num_tokens_to_remove = len(tokens) - MAX_TOKENS
      truncated_tokens = tokens[:-num_tokens_to_remove]
      combined_text = encoding.decode(truncated_tokens)


    return combined_prompt, relevant_history

In [ ]:
def test_final_prompt():
    """
    This function should:
    1. Define the final test prompt.
    2. Prepare the prompt using the prepare_prompt function.
    3. Call OpenAI to generate a response.
    4. Print the final test prompt, the chat history context referred to, and the final response.
    5. Validate manually if the final response references the correct context.
    """
    final_test_prompt = "How to learn Python"

    # Prepare the prompt using the prepare_prompt function
    prepared_prompt, context_referred = prepare_prompt(final_test_prompt)
    print(prepared_prompt,"promt")

    # Get the chat completion response from OpenAI
    response = model.generate_content(prepared_prompt)

    print(f"Final Test Prompt: {final_test_prompt}")
    print(f"Context Referred: {context_referred}")
    print(f"Final Test Prompt Response: {response.text}")

# Call the test function to generate the Final Test Prompt Response
test_final_prompt()